In [90]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_excel("faltas_treinamento_sujo.xlsx")

1. Higienização Básica<br/>
    a. Verifique valores nulos e tipos incorretos<br/>
    b. Corrija a coluna de datas inválidas<br/>
    c. Normalize os nomes dos turnos e dias da semana

In [91]:
# a)

print(df.isnull().sum())

## Verifica tipos gerais da coluna
print(df.dtypes)

## Verifica tipos de todos os dados da coluna
for coluna in df.columns:
   tipos = df[coluna].map(type).value_counts()
   print(f"\nColuna: {coluna}")
   print(tipos)


# b)

df['Data_Ultima_Falta'] = pd.to_datetime(df['Data_Ultima_Falta'], errors='coerce', format='%d/%m/%Y')

# c)

## Normalização de turnos
df['Turno'] = df['Turno'].str.strip()
df['Turno'] = df['Turno'].str.title()
turnos_tratados = {'Manha': 'Manhã'}
df['Turno'] = df['Turno'].replace(turnos_tratados)

## Normalização de dias da semana
df['Dia_da_Semana_Ultima_Falta'] = df['Data_Ultima_Falta'].dt.day_name()

dias_semana = {
    'Monday': 'Segunda-feira',
    'Tuesday': 'Terça-feira',
    'Wednesday': 'Quarta-feira',
    'Thursday': 'Quinta-feira',
    'Friday': 'Sexta-feira',
    'Saturday': 'Sábado',
    'Sunday': 'Domingo'
}

df['Dia_da_Semana_Ultima_Falta'] = df['Dia_da_Semana_Ultima_Falta'].map(dias_semana)

print(df['Dia_da_Semana_Ultima_Falta'])


Nome                           0
Departamento                   0
Turno                          0
Faltas                         0
Dia_da_Semana_Ultima_Falta     0
Data_Ultima_Falta              0
Evadiu                        14
dtype: int64
Nome                          object
Departamento                  object
Turno                         object
Faltas                         int64
Dia_da_Semana_Ultima_Falta    object
Data_Ultima_Falta             object
Evadiu                        object
dtype: object

Coluna: Nome
Nome
<class 'str'>    224
Name: count, dtype: int64

Coluna: Departamento
Departamento
<class 'str'>    224
Name: count, dtype: int64

Coluna: Turno
Turno
<class 'str'>    224
Name: count, dtype: int64

Coluna: Faltas
Faltas
<class 'int'>    224
Name: count, dtype: int64

Coluna: Dia_da_Semana_Ultima_Falta
Dia_da_Semana_Ultima_Falta
<class 'str'>    224
Name: count, dtype: int64

Coluna: Data_Ultima_Falta
Data_Ultima_Falta
<class 'str'>    224
Name: count, dtype: i

2. Remoção e Substituição<br/>
    a. Remova os registros duplicados<br/>
    b. Substitua os valores nulos da coluna Evadiu por “Desconhecido”<br/>
    c. Substitua os valores nulos da coluna Data_Ultima_Falta pela data mais comum

In [92]:
# a)

df = df.drop_duplicates()

# b)

df['Evadiu'].fillna("Desconhecido", inplace=True)

# c)

df['Data_Ultima_Falta'].fillna(df['Data_Ultima_Falta'].mode(), inplace=True)
df['Data_Ultima_Falta'].fillna("2024-03-24 00:00:00", inplace=True)
df['Dia_da_Semana_Ultima_Falta'].fillna("Domingo", inplace=True)

C:\Users\53144094889\AppData\Local\Temp\ipykernel_3736\4088120017.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Evadiu'].fillna("Desconhecido", inplace=True)
C:\Users\53144094889\AppData\Local\Temp\ipykernel_3736\4088120017.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a co

3. Correções e Detecção de Outliers<br/>
    a. Converta a coluna Faltas para tipo inteiro<br/>
    b. Substitua outliers de faltas por np.nan e depois pela mediana<br/>
    c. Crie um flag “Faltas_Suspeitas” para valores > 10   

In [93]:
# a)

df['Faltas'] = df['Faltas'].astype(int)

# b)

Q1 = df['Faltas'].quantile(0.25)
Q3 = df['Faltas'].quantile(0.75)
IQR = Q3 - Q1

limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

df['Faltas'] = df['Faltas'].apply(lambda x: np.nan if (x < limite_inferior or x > limite_superior) else x)

mediana_faltas = df['Faltas'].median()
df['Faltas'].fillna(mediana_faltas, inplace=True)

# c)

df['Faltas_Suspeitas'] = df['Faltas'].apply(lambda falta: "Sim" if falta > 10 else ("Não" if falta <= 10 else "Desconhecido"))


C:\Users\53144094889\AppData\Local\Temp\ipykernel_3736\1147105453.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Faltas'].fillna(mediana_faltas, inplace=True)


In [94]:
df.to_excel('faltas_treinamento_tratadas.xlsx', index=False)